In [11]:
%reload_ext autoreload
%autoreload 2

In [12]:
import json, pprint
from pathlib import Path
from toolz import take
import re

import findspark
findspark.init()
from pyspark.sql import SparkSession

In [13]:
import _scproxy
import _hashlib
_hashlib.__file__

'/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload/_hashlib.cpython-37m-darwin.so'

In [14]:
def try_import(name):
    import sys, os, csv
    from pathlib import Path
    from toolz import concatv
    import _hashlib
    cwd = Path()
    reader = sorted(concatv(
        [('sys', 'script', sys.argv[0], name)],
        [('sys', 'executable', sys.executable, name)],
        [('sys', 'path', sys.path, name)],
        [('path', 'cwd', str(cwd.absolute()), name)],
        [('path', '_hashlib', str(_hashlib.__file__), name)],
        sorted(('module', k, str(mod), name) for k, mod in sys.modules.items()),
        sorted(('environ', k, str(val), name) for k, val in dict(os.environ).items()),
    ))
    
    infile = cwd /f"{name}.csv"

    with infile.open('w') as fh:
        writer = csv.writer(fh)
        writer.writerow(['category','type','item', 'source'])
        for rec in reader:
            writer.writerow(rec)
    result = {'infile': str(infile),
            'record_cnt': len(reader)}
    print(f'>> result={result}')
    #import _scproxy
    #import numpy
    #import pandas

    return json.dumps(sys.path)

In [15]:
ret = try_import('notebook_local')
ret

>> result={'infile': 'notebook_local.csv', 'record_cnt': 1249}


'["/Users/wmcabee/opt/spark/python", "/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip", "/Users/wmcabee/opt/spark/python", "/private/var/folders/zg/vyq842cx20s66ytjhf0bl43dxzgtx0/T/spark-72b3bbe2-3a13-45c5-98a1-e750b3c54e10/userFiles-2d9fa871-86a0-4d0b-a05e-cb4ad281306a", "/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip", "/Users/wmcabee/_NBC/nbc_analysis/tests/test_app_io/clickstream_events", "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python37.zip", "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7", "/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload", "", "/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages", "/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/setuptools-39.1.0-py3.7.egg", "/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/pip-10.0.1-py3.7.egg", "/Users/wmcabee/_NBC/nbc_analysis/src", "/Use

In [16]:
spark = (SparkSession
        .builder
        .master("local")
        .appName("Import Test")
        .getOrCreate() )
df = spark.createDataFrame([{'iteration': 1} ])
rdd = df.rdd.map(lambda x: try_import('notebook_spark'))
ret = rdd.take(1)
pprint.pprint(ret)

['["/Users/wmcabee/_NBC/nbc_analysis/tests/test_app_io/clickstream_events", '
 '"/private/var/folders/zg/vyq842cx20s66ytjhf0bl43dxzgtx0/T/spark-72b3bbe2-3a13-45c5-98a1-e750b3c54e10/userFiles-2d9fa871-86a0-4d0b-a05e-cb4ad281306a", '
 '"/Users/wmcabee/opt/spark/python/lib/pyspark.zip", '
 '"/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip", '
 '"/Users/wmcabee/opt/spark-2.4.4/jars/spark-core_2.11-2.4.4.jar", '
 '"/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python37.zip", '
 '"/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7", '
 '"/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload", '
 '"/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages", '
 '"/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/setuptools-39.1.0-py3.7.egg", '
 '"/Users/wmcabee/_NBC/nbc_analysis/venv/lib/python3.7/site-packages/pip-10.0.1-py3.7.egg", '
 '"/Users/wmcabee/_

In [17]:
import pandas as pd

In [18]:
reader = Path().glob('cmd_*.csv')
reader = map(pd.read_csv,reader)
df = pd.concat(reader)
with pd.option_context('max_rows',None, 'max_columns', None, 'max_colwidth', 200):
    dx = df.query("category == 'environ'")
    #display(dx)

df = df.set_index(['category','type','source']).item.unstack()
df= df.sort_index()
df['match'] = df.fillna('na').cmd_local == df.fillna('na').cmd_spark
df = df[~df.match].copy()
DATA = df
#DATA

In [21]:
# ENVIRONMENT VARIABLES
df = DATA.reset_index()
df = df.query("category == 'environ'").drop(columns=['category','match'])
df.columns.name = None
df = df.set_index('type').sort_index()
with pd.option_context('max_rows',None, 'max_columns', None, 'max_colwidth', 400):
    display(df)

,cmd_local,cmd_spark
type,,
JAVA_MAIN_CLASS_64185,NaN,org.apache.spark.deploy.SparkSubmit
PYTHONHASHSEED,NaN,0
PYTHONPATH,/Users/wmcabee/opt/spark/python/lib/pyspark.zip:/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip:/Users/wmcabee/opt/spark-2.4.4/jars/spark-core_2.11-2.4.4.jar,/Users/wmcabee/opt/spark/python/lib/pyspark.zip:/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip:/Users/wmcabee/opt/spark-2.4.4/jars/spark-core_2.11-2.4.4.jar:/Users/wmcabee/opt/spark/python/lib/pyspark.zip:/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip:/Users/wmcabee/opt/spark-2.4.4/jars/spark-core_2.11-2.4.4.jar
PYTHONUNBUFFERED,NaN,YES
PYTHON_WORKER_FACTORY_SECRET,NaN,b270bdf171f1729de1969cd38f31c67df400c7c4fd01ff831ed70294f7882fb0
SPARK_CONF_DIR,NaN,/Users/wmcabee/opt/spark/conf
SPARK_ENV_LOADED,NaN,1
SPARK_LOCAL_DIRS,NaN,/private/var/folders/zg/vyq842cx20s66ytjhf0bl43dxzgtx0/T/blockmgr-dd900856-d7ab-4ad5-a606-b46601ce7542
SPARK_REUSE_WORKER,NaN,1


In [214]:
# Look at module differences
MODULES = DATA.xs('module',drop_level=False).copy()
df = MODULES.copy()
ds = df.cmd_local.str.extract(r"from '(.+)'")
df['cmd_local'] = ds.where(ds.notna(), df.cmd_local, axis=0)

ds = df.cmd_spark.str.extract(r"from '(.+)'")
df['cmd_spark'] = ds.where(ds.notna(), df.cmd_spark, axis=0)
with pd.option_context('max_rows',None, 'max_columns', None, 'max_colwidth', 200):
    display(df)

source                                             cmd_local  \
category type                                                  
module   __main__                           ./import_test.py   
         _bz2                                            NaN   
         _compat_pickle                                  NaN   
         _compression                                    NaN   
         _ctypes                                         NaN   
         _cython_0_29_13                                 NaN   
         _datetime                                       NaN   
         _decimal                                        NaN   
         _lsprof                                         NaN   
         _lzma                                           NaN   
         _pickle                                         NaN   
         _posixsubprocess                                NaN   
         _queue                                          NaN   
         _socket                                         NaN   
         _string                                         NaN   
         _struct                                         NaN   
         _weakrefset                                     NaN   
         argparse                                        NaN   
         array                                           NaN   
         atexit                                          NaN   
         base64                                          NaN   
         binascii                                        NaN   
         bz2                                             NaN   
         cProfile                                        NaN   
         calendar                                        NaN   
         copy                                            NaN   
         ctypes                                          NaN   
         ctypes._endian                                  NaN   
         cython_runtime                                  NaN   
         datetime                                        NaN   
         decimal                                         NaN   
         difflib                                         NaN   
         gc                                              NaN   
         gettext                                         NaN   
         grp                                             NaN   
         hmac                                            NaN   
         html                                            NaN   
         html.entities                                   NaN   
         imp                                             NaN   
         locale                                          NaN   
         logging                                         NaN   
         lzma                                            NaN   
         numbers                                         NaN   
         numpy                                           NaN   
         numpy.__config__                                NaN   
         numpy._distributor_init                         NaN   
         numpy._globals                                  NaN   
         numpy._pytesttester                             NaN   
         numpy.compat                                    NaN   
         numpy.compat._inspect                           NaN   
         numpy.compat.py3k                               NaN   
         numpy.core                                      NaN   
         numpy.core._add_newdocs                         NaN   
         numpy.core._asarray                             NaN   
         numpy.core._dtype                               NaN   
         numpy.core._dtype_ctypes                        NaN   
         numpy.core._exceptions                          NaN   
         numpy.core._internal                            NaN   
         numpy.core._methods                             NaN   
         numpy.core._multiarray_tests                    NaN   
         numpy.core._multiarray_

In [180]:
dx = DATA.copy()
idx = dx.index.difference(MODULES.index)
dx = dx.loc[idx].reset_index()
dx  = dx.query('category=="sys" and type=="path"')
dx = dx[['cmd_local','cmd_spark']].stack()
dx = dx.reset_index(0, drop=True)

def clean_item(val):
    val = re.sub(r"[',[\]]", '', val)
    return val
reader = ((source, clean_item(item)) for source, items in dx.items() for item in items.split())
#reader = take(10,reader)
reader = ((source, item, idx) for idx, (source, item) in enumerate(reader) if item != '')


reader = sorted(reader)
dx = pd.DataFrame.from_records(reader, columns=['source','items', 'idx'])
dx = dx.set_index(['items', 'source']).idx.sort_index().unstack()
dx['cmd_spark'] = dx.cmd_spark - dx.cmd_spark.min() -2
dx = dx.sort_values(['cmd_local', 'cmd_spark'], na_position='first')
with pd.option_context('max_rows',None, 'max_columns', None, 'max_colwidth', 200):
    display(dx)

source,cmd_local,cmd_spark
items,,
/private/var/folders/zg/vyq842cx20s66ytjhf0bl43dxzgtx0/T/spark-d26c492a-dda7-4b38-982d-d8698cd4e798/userFiles-33850be8-4e52-4969-897e-bba6e8e8879d,NaN,-1.0
/Users/wmcabee/opt/spark/python/lib/pyspark.zip,NaN,0.0
/Users/wmcabee/opt/spark-2.4.4/jars/spark-core_2.11-2.4.4.jar,NaN,2.0
/Users/wmcabee/opt/spark/python,0.0,NaN
/Users/wmcabee/opt/spark/python/lib/py4j-0.10.7-src.zip,1.0,1.0
/Users/wmcabee/_NBC/nbc_analysis/ana/import_error,2.0,-2.0
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python37.zip,3.0,3.0
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7,4.0,4.0
/usr/local/Cellar/python/3.7.4/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload,5.0,5.0


In [48]:
x = dx.index

In [51]:
dx.difference(MODULES.index)

MultiIndex([('environ',          'JAVA_MAIN_CLASS_52691'),
            ('environ',                         'OLDPWD'),
            ('environ',                            'PS1'),
            ('environ',                 'PYTHONHASHSEED'),
            ('environ',                     'PYTHONPATH'),
            ('environ',               'PYTHONUNBUFFERED'),
            ('environ',   'PYTHON_WORKER_FACTORY_SECRET'),
            ('environ',                 'SPARK_CONF_DIR'),
            ('environ',               'SPARK_ENV_LOADED'),
            ('environ',               'SPARK_LOCAL_DIRS'),
            ('environ',             'SPARK_REUSE_WORKER'),
            ('environ',            'SPARK_SCALA_VERSION'),
            ('environ',                              '_'),
            ('environ', '_PYSPARK_DRIVER_CONN_INFO_PATH'),
            (    'sys',                     'executable'),
            (    'sys',                         'script')],
           names=['category', 'type'])